In [169]:
import pandas as pd

from flatlib import const
from flatlib.chart import Chart
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib.object import GenericObject


from flatlib.tools import arabicparts
from flatlib.dignities import essential

from _astro_constants import AstrologicalConstants, AstrologicalPoints

pd.set_option('display.max_columns', 200)  # or 1000
pd.set_option('display.max_rows', 200)  # or 1000
pd.set_option('display.max_colwidth', 100)  # or 199

In [170]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [152]:
data = {'date':['2015/03/13', '2016/05/23', '2018/10/03'], 'utc_time':['17:00', '12:30', '00:15'], 'lon':['38n32', '08n07', '14s42'], 'lat':['8w54', '18e14','36w04']}
df = pd.DataFrame(data)
df

,date,utc_time,lon,lat
0,2015/03/13,17:00,38n32,8w54
1,2016/05/23,12:30,08n07,18e14
2,2018/10/03,00:15,14s42,36w04


In [153]:
df['charts'] = AstrologicalPoints.charts_calculate(df, 'date', 'utc_time', 'lon', 'lat')

In [154]:
df.charts

0    <flatlib.chart.Chart object at 0x7fc593fabee0>
1    <flatlib.chart.Chart object at 0x7fc593fca250>
2    <flatlib.chart.Chart object at 0x7fc593fcad30>
Name: charts, dtype: object

In [155]:
rulers_constants = AstrologicalConstants.rulers_constants
str(rulers_constants)

"['ASC', 'DESC', 'MC', 'IC', 'PARS_FORTUNA']"

In [156]:
necessary_constants = AstrologicalConstants.necessary_constants
str(necessary_constants)

"['SUN', 'MOON', 'SATURN', 'URANUS', 'NEPTUNE', 'PLUTO', 'CHIRON', 'NORTH_NODE', 'SOUTH_NODE']"

In [157]:
for obj in rulers_constants + necessary_constants:
    df[str.lower(obj)] = AstrologicalPoints.calculate_astro_objects(df.charts, obj)

In [158]:
rulers_col_names = AstrologicalPoints.rulers_col_names(rulers_constants)
rulers_col_names

['ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'ruler_pars_fortuna']

In [159]:
for name, obj in zip(rulers_col_names, rulers_constants):
    obj = str.lower(obj)
    df[name] = AstrologicalPoints.ruler_of_object(df[obj])
    df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)

In [160]:
antes_objects = AstrologicalConstants.antes_objects
str(antes_objects)

"['pars_fortuna', 'uranus', 'neptune', 'pluto', 'chiron']"

In [161]:
for obj in antes_objects:
    antes_name = 'antes_' + obj
    df[antes_name] = AstrologicalPoints.antes_objects(df, obj)

In [162]:
df

,date,utc_time,lon,lat,charts,asc,desc,mc,ic,pars_fortuna,sun,moon,saturn,uranus,neptune,pluto,chiron,north_node,south_node,ruler_asc,ruler_desc,ruler_mc,ruler_ic,ruler_pars_fortuna,antes_pars_fortuna,antes_uranus,antes_neptune,antes_pluto,antes_chiron
0,2015/03/13,17:00,38n32,8w54,<flatlib.chart.Chart object at 0x7fc593fabee0>,<Asc Virgo +03:27:30>,<Desc Pisces +03:27:30>,<MC Taurus +29:19:03>,<IC Scorpio +29:19:03>,<Pars Fortuna Gemini +03:03:00 +00:00:00>,<Sun Pisces +22:47:25 +00:59:51>,<Moon Sagittarius +22:22:54 +13:16:01>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Uranus Aries +15:06:57 +00:03:14>,<Neptune Pisces +07:50:55 +00:02:13>,<Pluto Capricorn +15:14:43 +00:01:02>,<Chiron Pisces +17:36:43 +00:03:43>,<North Node Libra +11:08:28 -00:03:11>,<South Node Aries +11:08:28 -00:03:11>,<Mercury Pisces +00:48:57 +01:29:49>,<Jupiter Leo +13:38:37 -00:04:45>,<Venus Aries +25:30:11 +01:12:41>,<Mars Aries +16:32:48 +00:45:18>,<Mercury Pisces +00:48:57 +01:29:49>,<Pars Fortuna Cancer +26:57:00 +00:00:00>,<Uranus Virgo +14:53:03 +00:03:14>,<Neptune Libra +22:09:05 +00:02:13>,<Pluto Sagittarius +14:45:17 +00:01:02>,<Chiron Libra +12:23:17 +00:03:43>
1,2016/05/23,12:30,08n07,18e14,<flatlib.chart.Chart object at 0x7fc593fca250>,<Asc Virgo +27:12:45>,<Desc Pisces +27:12:45>,<MC Gemini +27:30:28>,<IC Sagittarius +27:30:28>,<Pars Fortuna Aries +15:36:29 +00:00:00>,<Sun Gemini +02:47:58 +00:57:39>,<Moon Sagittarius +21:11:41 +12:19:41>,<Saturn Sagittarius +13:54:21 -00:04:19>,<Uranus Aries +22:47:40 +00:02:49>,<Neptune Pisces +11:54:55 +00:00:42>,<Pluto Capricorn +17:11:35 -00:00:57>,<Chiron Pisces +24:43:00 +00:01:47>,<North Node Virgo +18:00:29 -00:03:11>,<South Node Pisces +18:00:29 -00:03:11>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mars Sagittarius +01:25:22 -00:21:00>,<Pars Fortuna Virgo +14:23:31 +00:00:00>,<Uranus Virgo +07:12:20 +00:02:49>,<Neptune Libra +18:05:05 +00:00:42>,<Pluto Sagittarius +12:48:25 -00:00:57>,<Chiron Libra +05:17:00 +00:01:47>
2,2018/10/03,00:15,14s42,36w04,<flatlib.chart.Chart object at 0x7fc593fcad30>,<Asc Gemini +05:25:08>,<Desc Sagittarius +05:25:08>,<MC Pisces +07:39:39>,<IC Virgo +07:39:39>,<Pars Fortuna Leo +27:31:49 +00:00:00>,<Sun Libra +09:44:33 +00:59:03>,<Moon Cancer +17:37:52 +14:05:52>,<Saturn Capricorn +03:06:46 +00:02:33>,<Uranus Taurus +01:23:25 -00:02:14>,<Neptune Pisces +14:24:42 -00:01:27>,<Pluto Capricorn +18:45:26 +00:00:03>,<Chiron Pisces +29:41:06 -00:02:40>,<North Node Leo +02:19:59 -00:03:11>,<South Node Aquarius +02:19:59 -00:03:11>,<Mercury Libra +18:46:13 +01:38:51>,<Jupiter Scorpio +22:27:07 +00:11:21>,<Jupiter Scorpio +22:27:07 +00:11:21>,<Mercury Libra +18:46:13 +01:38:51>,<Sun Libra +09:44:33 +00:59:03>,<Pars Fortuna Taurus +02:28:11 +00:00:00>,<Uranus Leo +28:36:35 -00:02:14>,<Neptune Libra +15:35:18 -00:01:27>,<Pluto Sagittarius +11:14:34 +00:00:03>,<Chiron Libra +00:18:54 -00:02:40>


In [201]:
chart = df.charts.values[0]
chart

In [203]:
df.asc.values[0].id = 'Desc'

In [204]:
print(df.asc.values[0].id)

Desc


In [163]:
main_objects = AstrologicalConstants.main_objects
str(main_objects)

"['moon', 'ruler_asc', 'ruler_desc', 'pars_fortuna', 'ruler_pars_fortuna', 'ruler_mc', 'ruler_ic', 'sun', 'saturn', 'uranus', 'neptune', 'pluto', 'chiron', 'north_node', 'south_node']"

In [167]:
houses_objects = AstrologicalConstants.houses_objects
str(houses_objects)

"['asc', 'desc', 'mc', 'ic']"

In [199]:
antes_columns = []
for col in df.columns:
    antes_column = co
    

"['pars_fortuna', 'uranus', 'neptune', 'pluto', 'chiron']"

In [ ]:
for obj in houses_objects + main_objects:

In [173]:
tmp = AstrologicalPoints.chart_unique_ids(df, main_objects)

In [175]:
tmp[0]

0    Moon
1    Moon
2    Moon
Name: moon, dtype: object

In [196]:
unique_ids = df[main_objects].apply(lambda x: list(set([str.upper(i.id) for i in x])), axis=1)

In [197]:
len(unique_ids[2])

12

In [198]:
unique_ids[1]

['SUN',
 'URANUS',
 'NORTH NODE',
 'PARS FORTUNA',
 'MERCURY',
 'SATURN',
 'PLUTO',
 'NEPTUNE',
 'MOON',
 'MARS',
 'SOUTH NODE',
 'JUPITER',
 'CHIRON']